In [34]:
from bs4 import BeautifulSoup
import pathlib
import requests as req
import pandas as pd

In [4]:
url = 'https://www.rotowire.com/basketball/nba-lineups.php'

In [5]:
resp = req.get(url)

In [8]:
soup = BeautifulSoup(resp.text)

In [33]:
# soup.head()

In [14]:
lineups_html = soup.find('div', attrs={'class': 'lineups'})

In [39]:
type(lineups_html)

bs4.element.Tag

In [36]:
raw_filename = '/Users/johnschroeder/Documents/gambling_stuff/data/01_raw/nba_lineups/2021/03/20/rotowire.html'

In [40]:
with open(raw_filename, 'w', encoding='utf-8') as fp:
    fp.write(str(lineups_html))

In [41]:
with open(raw_filename, 'r') as fp:
    soup = BeautifulSoup(fp.read())

In [45]:
soup.find_all()[0].find_all('div', attrs={'class': 'lineup__teams'})

1

In [ ]:
lineups_html

In [91]:
matchups_html = [
    x for x 
    in lineups_html.find_all('div', attrs={'class': 'lineup__box'})
    if 'lineup__teams' in str(x)
    ]

In [111]:
lineup_players_list = []
for matchup_html in matchups_html:
    matchup_teams_html = matchup_html.find('div', attrs={'class': 'lineup__teams'})
    matchup = ''.join([x.text for x in matchup_teams_html.find_all('div')])
    matchup_teams = [
        {
            'TEAM_ABBREVIATION': x.find('div').text,
            'HOME_AWAY': x.get('class')[-1],
            'MATCHUP': matchup,
        }
        for x in matchup_teams_html.find_all('a')
    ]
    print(matchup_teams)
    for team in matchup_teams:
        abbrev = team.get('TEAM_ABBREVIATION')
        home_away = team.get('HOME_AWAY')
        lineup_raw = [x for x in matchup_html.find_all('ul') if home_away in str(x)][0]
        players_raw = [x for x in lineup_raw.find_all('li') if 'lineup__player' in str(x)]
        for player in players_raw:
            player_dict = {
                'TEAM_ABBREVIATION': abbrev,
                'HOME_AWAY': home_away,
                'MATCHUP': matchup,
                'TO_PLAY_DESC': player.get('title'),
                'PLAYER_NAME': player.find('a').get('title'),
                'NAME_SHORT': player.find('a').text,
                'STARTING_POSITION': player.find('div', attrs={'class', 'lineup__pos'}).text,
            }
            lineup_players_list.append(player_dict)

[{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'MATCHUP': 'ATLLAL'}, {'TEAM_ABBREVIATION': 'LAL', 'HOME_AWAY': 'is-home', 'MATCHUP': 'ATLLAL'}]
[{'TEAM_ABBREVIATION': 'SAC', 'HOME_AWAY': 'is-visit', 'MATCHUP': 'SACPHI'}, {'TEAM_ABBREVIATION': 'PHI', 'HOME_AWAY': 'is-home', 'MATCHUP': 'SACPHI'}]
[{'TEAM_ABBREVIATION': 'GS', 'HOME_AWAY': 'is-visit', 'MATCHUP': 'GSMEM'}, {'TEAM_ABBREVIATION': 'MEM', 'HOME_AWAY': 'is-home', 'MATCHUP': 'GSMEM'}]
[{'TEAM_ABBREVIATION': 'SA', 'HOME_AWAY': 'is-visit', 'MATCHUP': 'SAMIL'}, {'TEAM_ABBREVIATION': 'MIL', 'HOME_AWAY': 'is-home', 'MATCHUP': 'SAMIL'}]
[{'TEAM_ABBREVIATION': 'CHA', 'HOME_AWAY': 'is-visit', 'MATCHUP': 'CHALAC'}, {'TEAM_ABBREVIATION': 'LAC', 'HOME_AWAY': 'is-home', 'MATCHUP': 'CHALAC'}]


In [114]:
lineup_df = pd.DataFrame(lineup_players_list).drop_duplicates()

In [115]:
lineup_df.to_csv('/Users/johnschroeder/Documents/gambling_stuff/data/02_curated/nba_lineups/2021/03/20/rotowire.csv', index=False)

In [109]:
lineup_df.head()

,TEAM_ABBREVIATION,HOME_AWAY,MATCHUP,TO_PLAY_DESC,PLAYER_NAME,NAME_SHORT,STARTING_POSITION
0,ATL,is-visit,ATLLAL,Very Likely To Play,Trae Young,Trae Young,PG
1,ATL,is-visit,ATLLAL,Very Likely To Play,Kevin Huerter,Kevin Huerter,SG
2,ATL,is-visit,ATLLAL,Very Likely To Play,Tony Snell,Tony Snell,SF
3,ATL,is-visit,ATLLAL,Very Likely To Play,John Collins,John Collins,PF
4,ATL,is-visit,ATLLAL,Very Likely To Play,Clint Capela,Clint Capela,C


In [107]:
lineup_players_list[:20]

[{'TEAM_ABBREVIATION': 'ATL',
  'HOME_AWAY': 'is-visit',
  'MATCHUP': 'ATLLAL',
  'TO_PLAY_DESC': 'Very Likely To Play',
  'PLAYER_NAME': 'Trae Young',
  'NAME_SHORT': 'Trae Young',
  'STARTING_POSITION': 'PG'},
 {'TEAM_ABBREVIATION': 'ATL',
  'HOME_AWAY': 'is-visit',
  'MATCHUP': 'ATLLAL',
  'TO_PLAY_DESC': 'Very Likely To Play',
  'PLAYER_NAME': 'Kevin Huerter',
  'NAME_SHORT': 'Kevin Huerter',
  'STARTING_POSITION': 'SG'},
 {'TEAM_ABBREVIATION': 'ATL',
  'HOME_AWAY': 'is-visit',
  'MATCHUP': 'ATLLAL',
  'TO_PLAY_DESC': 'Very Likely To Play',
  'PLAYER_NAME': 'Tony Snell',
  'NAME_SHORT': 'Tony Snell',
  'STARTING_POSITION': 'SF'},
 {'TEAM_ABBREVIATION': 'ATL',
  'HOME_AWAY': 'is-visit',
  'MATCHUP': 'ATLLAL',
  'TO_PLAY_DESC': 'Very Likely To Play',
  'PLAYER_NAME': 'John Collins',
  'NAME_SHORT': 'John Collins',
  'STARTING_POSITION': 'PF'},
 {'TEAM_ABBREVIATION': 'ATL',
  'HOME_AWAY': 'is-visit',
  'MATCHUP': 'ATLLAL',
  'TO_PLAY_DESC': 'Very Likely To Play',
  'PLAYER_NAME': 'Cli

In [92]:
[x.find_all('div', attrs={'class': 'lineup__teams'}) for x in matchups_html]

[[<div class="lineup__teams">
  <a class="lineup__team is-visit" href="/basketball/depth-chart.php?team=ATL">
  <img alt="ATL" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/basketball/100ATL.png?v=2"/>
  <div class="lineup__abbr">ATL</div>
  </a>
  <a class="lineup__team is-home" href="/basketball/depth-chart.php?team=LAL">
  <img alt="LAL" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/basketball/100LAL.png?v=2"/>
  <div class="lineup__abbr">LAL</div>
  </a>
  </div>],
 [<div class="lineup__teams">
  <a class="lineup__team is-visit" href="/basketball/depth-chart.php?team=SAC">
  <img alt="SAC" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/basketball/100SAC.png?v=2"/>
  <div class="lineup__abbr">SAC</div>
  </a>
  <a class="lineup__team is-home" href="/basketball/depth-chart.php?team=PHI">
  <img alt="PHI" class="lineup__logo" src="https://content.rotowire.com/images/teamlogo/basketball/100PHI.png?v=2"/>
  <div c

In [29]:
one_lineup = lineups_raw[0]

In [ ]:
game_lineup = []

In [ ]:
player_dict = {
    'TEAM_ABBREVIATION': '',
    'HOME_AWAY': '',
    'TO_PLAY_DESC': '',
    'PLAYER_NAME': '',
    'NAME_SHORT': '',
    'STARTING_POSITION': '',
}

In [77]:
lineup_teams

[{'TEAM_ABBREVIATION': 'ATL', 'IS_HOME': 0},
 {'TEAM_ABBREVIATION': 'LAL', 'IS_HOME': 1}]

In [76]:
one_lineup.find_all('ul')[0].find_all('li')

[<li class="lineup__status is-expected">
 <div class="dot is-medium is-yellow" style="margin-right:5px;"></div>Expected Lineup
             </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">PG</div>
 <a href="/basketball/player.php?id=4369" title="Trae Young">Trae Young</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">SG</div>
 <a href="/basketball/player.php?id=4449" title="Kevin Huerter">Kevin Huerter</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">SF</div>
 <a href="/basketball/player.php?id=3465" title="Tony Snell">Tony Snell</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">PF</div>
 <a href="/basketball/player.php?id=4159" title="John Collins">John Collins</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <

In [47]:
one_lineup_players = [x for x in one_lineup.find_all('li') if 'lineup__player' in str(x)]

In [61]:
for i, j in enumerate(one_lineup.find_all('li')):
    if 'Expected Lineup' in j.text:
        print(i)

0
11


In [60]:
one_lineup.find_all('li')   

[<li class="lineup__status is-expected">
 <div class="dot is-medium is-yellow" style="margin-right:5px;"></div>Expected Lineup
             </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">PG</div>
 <a href="/basketball/player.php?id=4369" title="Trae Young">Trae Young</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">SG</div>
 <a href="/basketball/player.php?id=4449" title="Kevin Huerter">Kevin Huerter</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">SF</div>
 <a href="/basketball/player.php?id=3465" title="Tony Snell">Tony Snell</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <div class="lineup__pos">PF</div>
 <a href="/basketball/player.php?id=4159" title="John Collins">John Collins</a>
 </li>,
 <li class="lineup__player is-pct-play-100" title="Very Likely To Play">
 <

In [ ]:
lineup_teams = []

In [63]:
lineup_teams_raw = one_lineup.find('div', attrs={'class': 'lineup__teams'})

In [ ]:
lineup

In [78]:
lineup_teams = []
for team in lineup_teams_raw.find_all('a'):
    team_abbrev = team.find('div').text
    home_away = team.get('class')[-1]
    team_dict = {
        'TEAM_ABBREVIATION': team_abbrev,
        'HOME_AWAY': home_away
    }
    lineup_teams.append(team_dict)

In [79]:
print(lineup_teams)

[{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit'}, {'TEAM_ABBREVIATION': 'LAL', 'HOME_AWAY': 'is-home'}]


In [84]:
[lineup for lineup in one_lineup.find_all('ul') if 'is-visit' in str(lineup)][0]

<ul class="lineup__list is-visit">
<li class="lineup__status is-expected">
<div class="dot is-medium is-yellow" style="margin-right:5px;"></div>Expected Lineup
            </li>
<li class="lineup__player is-pct-play-100" title="Very Likely To Play">
<div class="lineup__pos">PG</div>
<a href="/basketball/player.php?id=4369" title="Trae Young">Trae Young</a>
</li>
<li class="lineup__player is-pct-play-100" title="Very Likely To Play">
<div class="lineup__pos">SG</div>
<a href="/basketball/player.php?id=4449" title="Kevin Huerter">Kevin Huerter</a>
</li>
<li class="lineup__player is-pct-play-100" title="Very Likely To Play">
<div class="lineup__pos">SF</div>
<a href="/basketball/player.php?id=3465" title="Tony Snell">Tony Snell</a>
</li>
<li class="lineup__player is-pct-play-100" title="Very Likely To Play">
<div class="lineup__pos">PF</div>
<a href="/basketball/player.php?id=4159" title="John Collins">John Collins</a>
</li>
<li class="lineup__player is-pct-play-100" title="Very Likely To

In [86]:
lineups = one_lineup.find_all('ul')

In [90]:
for team in lineup_teams[:1]:
    abbrev = team.get('TEAM_ABBREVIATION')
    home_away = team.get('HOME_AWAY')
    team_players_raw = [x for x in lineups if home_away in str(x)][0]
    team_players_tags = [
        x for x in team_players_raw.find_all('li') 
        if 'lineup__player' in str(x)
        ]
    for player in team_players_tags:
        player_dict = {
            'TEAM_ABBREVIATION': abbrev,
            'HOME_AWAY': home_away,
            'TO_PLAY_DESC': player.get('title'),
            'PLAYER_NAME': player.find('a').get('title'),
            'NAME_SHORT': player.find('a').text,
            'STARTING_POSITION': player.find('div', attrs={'class', 'lineup__pos'}).text,
        }
        print(player_dict)

{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Trae Young', 'NAME_SHORT': 'Trae Young', 'STARTING_POSITION': 'PG'}
{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Kevin Huerter', 'NAME_SHORT': 'Kevin Huerter', 'STARTING_POSITION': 'SG'}
{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Tony Snell', 'NAME_SHORT': 'Tony Snell', 'STARTING_POSITION': 'SF'}
{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'John Collins', 'NAME_SHORT': 'John Collins', 'STARTING_POSITION': 'PF'}
{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Clint Capela', 'NAME_SHORT': 'Clint Capela', 'STARTING_POSITION': 'C'}
{'TEAM_ABBREVIATION': 'ATL', 'HOME_AWAY': 'is-visit', 'TO_PLAY_DESC': 'Very Unlikely To Play', 'PLAYER

In [53]:
for player in one_lineup_players:

    print(player_dict)

{'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Trae Young', 'NAME_SHORT': 'Trae Young', 'STARTING_POSITION': 'PG'}
{'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Kevin Huerter', 'NAME_SHORT': 'Kevin Huerter', 'STARTING_POSITION': 'SG'}
{'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Tony Snell', 'NAME_SHORT': 'Tony Snell', 'STARTING_POSITION': 'SF'}
{'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'John Collins', 'NAME_SHORT': 'John Collins', 'STARTING_POSITION': 'PF'}
{'TO_PLAY_DESC': 'Very Likely To Play', 'PLAYER_NAME': 'Clint Capela', 'NAME_SHORT': 'Clint Capela', 'STARTING_POSITION': 'C'}
{'TO_PLAY_DESC': 'Very Unlikely To Play', 'PLAYER_NAME': 'Kris Dunn', 'NAME_SHORT': 'K. Dunn', 'STARTING_POSITION': 'G'}
{'TO_PLAY_DESC': 'Very Unlikely To Play', 'PLAYER_NAME': 'Brandon Goodwin', 'NAME_SHORT': 'B. Goodwin', 'STARTING_POSITION': 'G'}
{'TO_PLAY_DESC': 'Very Unlikely To Play', 'PLAYER_NAME': "De'Andre Hunter", 'NAME_SHORT': 'D. Hunter', 'STARTING_POS